Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [17]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import statsmodels.formula.api as smf

In [2]:
data = pd.read_table('botswana.tsv')
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [3]:
data['religion'].value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [4]:
data.dropna().shape[0]

1834

В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак x_1x 
1
​	
  на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

создать новый бинарный признак
x_2 =
{1,0,x1='не применимо',иначе;
x 
2
​	
 ={ 
1,
0,
​	
  
x 
1
​	
 =’не применимо’,
иначе;
​	
 
заменить "не применимо" в x_1x 
1
​	
  на произвольную константу cc, которая среди других значений x_1x 
1
​	
  не встречается.
Теперь, когда мы построим регрессию на оба признака и получим модель вида
y=\beta_0 + \beta_1 x_1 + \beta_2 x_2,
y=β 
0
​	
 +β 
1
​	
 x 
1
​	
 +β 
2
​	
 x 
2
​	
 ,
на тех объектах, где x_1x 
1
​	
  было измерено, регрессионное уравнение примет вид
y=\beta_0 + \beta_1 x,
y=β 
0
​	
 +β 
1
​	
 x,
а там, где x_1x 
1
​	
  было "не применимо", получится
y=\beta_0 + \beta_1 c + \beta_2.
y=β 
0
​	
 +β 
1
​	
 c+β 
2
​	
 .
Выбор cc влияет только на значение и интерпретацию \beta_2β 
2
​	
 , но не \beta_1β 
1
​	
 .

Давайте используем этот метод для обработки пропусков в agefm и heduc.

Создайте признак nevermarr, равный единице там, где в agefm пропуски.
Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице XX будет мультиколлинеарность.
Замените NaN в признаке agefm на c_{agefm}=0c 
agefm
​	
 =0.
У объектов, где nevermarr = 1, замените NaN в признаке heduc на c_{heduc_1}=-1c 
heduc 
1
​	
 
​	
 =−1 (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).
Сколько осталось пропущенных значений в признаке heduc?

In [5]:
c = pd.Series([1 if data.loc[i, 'agefm'] != data.loc[i, 'agefm'] else 0 for i in range(data.shape[0])])

In [6]:
data = data.assign(nevermarr=c)

In [7]:
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [8]:
data.drop(columns=['evermarr'], inplace=True)
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [9]:
data['agefm'].fillna(0., inplace=True)
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [10]:
data['heduc'][data['heduc'].isnull() & data['nevermarr'] == 1] = -1
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [11]:
data['heduc'].isnull().value_counts()

False    4238
True      123
Name: heduc, dtype: int64

Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения (c_{idlnchld}=-1c 
idlnchld
​	
 =−1, c_{heduc_2}=-2c 
heduc 
2
​	
 
​	
 =−2 (значение -1 мы уже использовали), c_{usemeth}=-1c 
usemeth
​	
 =−1).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [12]:
data['idlnchld_noans'] = 0
data['heduc_noans'] = 0
data['usemeth_noans'] = 0
data.loc[data['idlnchld'].isnull(), 'idlnchld_noans'] = 1
data.loc[data['heduc'].isnull(), 'heduc_noans'] = 1
data.loc[data['usemeth'].isnull(), 'usemeth_noans'] = 1
data.loc[data['idlnchld'].isnull()] = -1
data.loc[data['heduc'].isnull()] = -2
data.loc[data['usemeth'].isnull()] = -1
data.shape[0] * data.shape[1]

78498

In [14]:
data.dropna(axis=0, inplace=True)
data.shape[0]

4354

Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации R^2R 
2
 ? Округлите до трёх знаков после десятичной точки.

Если код из примера у вас не воспроизводится:

убедитесь, что вы сделали так:

возможно, вам нужно обновить библиотеку patsy; выполните в командной строке

In [19]:
model = smf.ols('ceb ~ ' + ' + '.join(data.columns[1:]), data=data)
model_fit = model.fit()
print(model_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.702
Model:                            OLS   Adj. R-squared:                  0.701
Method:                 Least Squares   F-statistic:                     567.1
Date:                Sun, 23 Aug 2020   Prob (F-statistic):               0.00
Time:                        23:48:41   Log-Likelihood:                -7460.2
No. Observations:                4354   AIC:                         1.496e+04
Df Residuals:                    4335   BIC:                         1.508e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0